<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Sentiment/Review_Sentiment_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import requests
import json

def sentiment(review):
  files = {
        'text': (None, '%s' % review),
                 #'["Come on,lets play together,Team performed well overall."]'),
        'api_key': (None, 'F5vtfGFDxwA2LXPAgZHY8Ib9WiHXUccN7599XCf6ggc'),
  }
  response = requests.post('https://apis.paralleldots.com/v4/sentiment_batch', files=files)

  f = open("response.json", "w")
  f.write(json.dumps(response.json(), indent=4))
  f.close()
  data = open("response.json", "r").read()
  data_parsed = json.loads(data)

  risp = data_parsed['sentiment'][0]

  massimo = max(risp['positive'],risp['negative'],risp['neutral'])

  if risp['negative'] == massimo:
    return 'negative' + ' (' + '%s' % risp['negative'] + ')'

  if risp['positive'] == massimo:
    return 'positive' + ' (' + '%s' % risp['positive'] + ')'

  if risp['neutral'] == massimo:
    return 'neutral' + ' (' + '%s' % risp['neutral'] + ')'

In [23]:
#sentiment di random, TO DO
#Da cambiare 
#nome_ristorante_sort
#users_restaurant_id = id del file csv del ristorante e nome del file
#cartella dove metto il csv (folder_id)

# Se sono i PRIMI 30

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import pandas as pd
import csv
import time

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nome_ristorante = 'TupleoHoney'
# DOWNLOAD merge.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Sentiment > Random_Reviews > (nomeRistorante)_merge.csv)
#File dove ci sono id e recensione
restaurant_merge_id = '1aOTHWSNdkjzW7EyyaomdXxgnzjXdqN7S'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': restaurant_merge_id})
download.GetContentFile('%s' % nome_ristorante + '_merge.csv')

user_review = pd.read_csv('%s' % nome_ristorante + '_merge.csv')

#creo e apro un file .csv per scrivere
file_parsed = open('%s' % nome_ristorante + '_Random_ReviewSentiment.csv', 'a')
    
#creo il writer
writer = csv.writer(file_parsed)

#creo header del csv
header = ['id', 'review_sentiment']
writer.writerow(header)

cnt = 1
api_number = 0

for i, rev in user_review.iterrows():
  #10% per ogni ristorante
  #if api_number < 300:
    if cnt < 30 :
      id = rev['user_id']
      response = sentiment([rev['text_review']])
      writer.writerow(['%s' % id, '%s' % response])
      cnt = cnt + 1
      api_number = api_number +1
    else:
      time.sleep(61)
      cnt =1
    else: 
      break

#salvo e chiudo il file .csv
file_parsed.close()

#folder_id si trova in data > Sentiment > (nomeRistorante)
#folder_id = '1etJ6Nt87YIEcYrFO0x-PLoYy612ez-5e'    #cartella dove metto il csv
#file_metadata = {'title': '%s' % nome_ristorante_sort + '_ReviewSentiment.csv',"parents":[{"id": folder_id,"kind":"drive#childList"}]}
#folder = drive.CreateFile(file_metadata)
#folder.SetContentFile('%s' % nome_ristorante_sort + '_ReviewSentiment.csv') #The contents of the file
#folder.Upload()

SyntaxError: ignored